# **Problem 1** #

## 1. ##
Proteins labelled with Cy5 -> Cy5 has excitation peak at ~ 649 nm, and emission peak at ~ 667 nm.

DNA labelled with Cy3 -> Cy3 has excitation peak at ~ 554 nm, and emission peak at ~ 566 nm.

<img src="setup.png" alt="setup" style="width:450px;"/> From "Fluorescence Correlation Spectroscopy
An Introduction to its Concepts and Applications", Petra Schwille and Elke Haustein

**Sample:**
A dilute solution (nanomolar range) containing the two species is prepared on a coverslip or within a sealed chamber. The low concentration is crucial for ensuring that only a few molecules reside in the focal volume at any given time.

**Objective:**
The objective lens focuses the beams to a small region and collects light emitted from this region. The objective lens must have a high numerical aperture (ideally NA > 0.9) in order to focus the laser light to a sufficiently small focal volume to ensure the number of simultaneously detected particles is not too large. Water immersion objectives are often used.

**Lasers:**
Either a single dual-channel laser, or 2 lasers, with wavelengths around the excitation peaks of the fluorophores (649 nm, 554 nm) will be used to excite the sample. The rest of the setup will assume the use of 2 lasers. Depending on the diameter and quality of the laser beams, beam expanders may be used to increase beam diameter so that the back aperture of the objective is more completely filled, resulting in a smaller focal volume.

**Dichroic mirrors:**
A dichroic mirror that reflects the 649 nm beam and transmits the 554 nm beam will be used to combine the two beams, before reflecting off a second dichroic mirror that reflects the excitation beams but transmits the emission wavelengths. This dichroic mirror will direct the beams toward the back aperture of the objective. After excitation and emission has occured, after the emission beams have passed through the second dichroic mirror, a final dichroic mirror is used that transmits the 566nm beam but reflects the 667 nm beam. This splits the 2 emission beams which are later collected and processed.

**Emission filters:**
Emission filters are used to selectively transmit the emitted light from the fluorophores to increase the signal to noise ratio. There will be one emission filter before the collection lens for the isolated 566 nm beam and one before the collection lens for the isolated 667 nm beam.

**Detectors + Lenses:**
The collection lenses focus the emission beams onto detectors. Pinholes are placed on the conjugate image plane (the focal point of the collection lenses corresponding to the location of the focal volume) to provide axial resolution by blocking unfocused light form below or on top of the focal volume. The detectors are incredibly sensitive, usually an avalanche photodiode or a photomultiplier with single photon sensitivity.

## 2. ##
The emission filters should have transmission wavlengths ~ 566 nm for the Cy3 emission beam, and ~ 667 nm for the Cy5 emission beam. These values are chosen because the purpose of the emission filters is to reduce the amount of photons hitting the detector that do not originate from the fluorophores. By selectively transmitting the emission wavelenth we can increase the signal to noise ratio by removing other sources of light from the sample.

## 3. ##
$$ D = \frac{k_b  T}{6 \pi \eta r}, \quad T \approx 293.15 [K], \quad \eta_{\text{water}} \approx 1E-3 [Pa \cdot s] \text{ from Wikipedia}$$
$$ \therefore r_{protein} = 3 [nm] \implies D_{protein} = 7.15E-11 [m^2 / s], \quad r_{DNA} = 200 [nm] \implies D_{DNA} = 1.07E-12 [m^2 / s]$$

## 4. ##
